In [1]:
# Set your API key to a variable

import os
from dotenv import load_dotenv, find_dotenv
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)
openai_api_key = os.environ["OPENAI_API_KEY"]

# Import the required packages
import langchain
import time
from openai import RateLimitError
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.document_loaders import UnstructuredHTMLLoader
from langchain_openai import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import UnstructuredHTMLLoader

In [2]:
#Load the HTML as a LangChain document loader
file_path ="data\mg-zs-warning-messages.html"
loader = UnstructuredHTMLLoader(file_path)
documents = loader.load()
#print(documents)
for doc in documents:
    print(doc.page_content)

Warning Message Procedure Cruise Control Fault Indicates that the cruise control system has detected a fault. Please consult an MG Authorised Repairer as soon as possible. Active Speed Limiter Fault Indicates that the active speed limit system has detected a fault. Contact an MG Authorised Repairer as soon as possible. Engine Coolant Temperature High High engine coolant temperature could result in severe damage. As soon as conditions permit, safely stop the vehicle and switch off the engine and contact an MG Authorised Repairer immediately. Engine Coolant Temperature Sensor Fault Indicates that the engine coolant temperature sensor has failed. As soon as conditions permit, safely stop the vehicle and switch off the engine and contact an MG Authorised Repairer immediately.

Warning Message Procedure Low Oil Pressure Indicates that the oil pressure is too low, which may result in severe engine damage. As soon as safety permits, stop the car, switch off the engine and check the engine oil

In [3]:
#Step 1: Splitting an HTML document into smaller 500-character chunks with a 50-character overlap for easier processing.
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)
#print(chunks)
for doc in documents:
    print(doc.page_content)

Warning Message Procedure Cruise Control Fault Indicates that the cruise control system has detected a fault. Please consult an MG Authorised Repairer as soon as possible. Active Speed Limiter Fault Indicates that the active speed limit system has detected a fault. Contact an MG Authorised Repairer as soon as possible. Engine Coolant Temperature High High engine coolant temperature could result in severe damage. As soon as conditions permit, safely stop the vehicle and switch off the engine and contact an MG Authorised Repairer immediately. Engine Coolant Temperature Sensor Fault Indicates that the engine coolant temperature sensor has failed. As soon as conditions permit, safely stop the vehicle and switch off the engine and contact an MG Authorised Repairer immediately.

Warning Message Procedure Low Oil Pressure Indicates that the oil pressure is too low, which may result in severe engine damage. As soon as safety permits, stop the car, switch off the engine and check the engine oil

In [4]:
import time
from openai import RateLimitError
from langchain.vectorstores import Chroma


def create_vector_store_with_retries(chunks, embedding, max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            vector_store = Chroma.from_documents(chunks, embedding)
            return vector_store
        except RateLimitError as e:
            print(e.message)
            retries += 1
            wait_time = 2 ** retries  # Exponential backoff
            print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
    raise Exception("Exceeded maximum retries")

# Step 2: Save the component embeddings to a vector store with retries
embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)


In [ ]:
#Creating Vector Store with Chroma
vector_store = Chroma.from_documents(chunks, embedding)
print(vector_store)

In [12]:
#Initializing Retriever from Vector Store
retriever = vector_store.as_retriever()

In [13]:
# Setting Up Car Manual Q&A with Prompt Template and Chat Model
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI

template = """Based on the car manual content below, answer the question.
Context: {context}
Question: {question}
Answer:"""

# Create the prompt template
prompt = PromptTemplate(input_variables=["context", "question"], template=template)

# Example usage
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [14]:
# Building the Retrieval-Augmented Generation (RAG) Chain
from langchain.schema.runnable import RunnablePassthrough

rag_chain = ({"context": retriever, "question": RunnablePassthrough()} | prompt | model)

In [15]:
query = "The Gasoline Particular Filter Full warning has appeared. What does this mean and what should I do about it?"
answer = rag_chain.invoke(query)
print(answer)

content='The Gasoline Particular Filter Full warning indicates that the gasoline particular filter is full. It is recommended to consult an MG Authorised Repairer as soon as possible to address this issue.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 499, 'total_tokens': 536, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-d9cd3cc8-fa86-44fb-9ce3-9f13e1f5d95d-0' usage_metadata={'input_tokens': 499, 'output_tokens': 37, 'total_tokens': 536, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
